# Batch processing pipeline for rate extraction only
1. Extract & cut spike trains
2. Compute firing rates

<u>Notes</u>:
- Batch processing of list of campaign paths
- By default, only excitatory spike trains and firing rates will be extracted
- BUT: Neuron info table and adjacency matrix will always contain all neurons

In [ ]:
import sys
sys.path.append('../../../library')

import os
from extract import run_extraction  # Requires bluepy kernel (BbpWorkflowKernel)
from preprocess import create_empty_data_store
from preprocess import run_rate_extraction, merge_rates_to_h5_data_store

In [ ]:
# syn_class = 'EXC'
# syn_class = 'INH'
syn_class = 'ALL'

working_dir_name = 'working_dir'  # Default name
if syn_class != 'EXC':
    working_dir_name = f'{working_dir_name}_{syn_class.lower()}'
print(f'Working dir: "{working_dir_name}" ({syn_class} spike trains)')

In [ ]:
# path_list = ['/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_Baseline',
#              '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewired_mc2EE_Order1',
#              '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewired_mc2EE_Order2',
#              '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewired_mc2EE_Order3',
#              '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewired_mc2EE_Order4',
#              '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewired_mc2EE_Order5']

# path_list = ['/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipRemoval_Unstruct-0',
#              '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipRemoval_Unstruct-1',
#              '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipRemoval_Unstruct-2',
#              '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipRemoval_Unstruct-3',
#              '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipRemoval_StructDim56',
#              '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipRemoval_StructDim56_456',
#              '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_RecipRemoval_StructDim456']

# path_list = ['/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnAdd_RecipStruct0x2',
#              '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnAdd_RecipStruct0x4',
#              '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnAdd_Control0x2',
#              '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnAdd_Control0x4']

path_list = ['/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewireEnhanced100K',
             '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewireEnhanced200K',
             '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewireEnhanced300K',
             '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewireEnhanced400K',
             '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewireEnhanced500K',
             '/gpfs/bbp.cscs.ch/data/scratch/proj9/bisimplices/simulations/BlobStimReliability_O1v5-SONATA_ConnRewireEnhanced670K']


In [ ]:
for idx, campaign_path in enumerate(path_list):
    print(f'PROCESSING {idx + 1}/{len(path_list)}: ../{os.path.split(campaign_path)[-1]}')

    # Extract & cut spike trains
    sim_paths, working_dir = run_extraction(campaign_path, working_dir_name=working_dir_name, syn_class=syn_class)
    num_sims = len(sim_paths)

    # Create empty data store without filtered spike signals
    create_empty_data_store(working_dir, data_store_name='processed_data_store')

    # Run rate extraction
    spike_file_names = [f'raw_spikes_{syn_class.lower()}_{idx}.npy' for idx in range(num_sims)]

    run_rate_extraction(working_dir, spike_file_names, syn_class=syn_class, pool_size=30)

    # Merge individual rate files into .h5 data store
    tmp_file_names = [f'firing_rates_{syn_class.lower()}_{idx}__tmp__.npz' for idx in range(num_sims)]

    h5_file = merge_rates_to_h5_data_store(working_dir, tmp_file_names, data_store_name='processed_data_store', do_overwrite=False)

__HOW TO LOAD PROCESSED SPIKE SIGNALS, META-INFO, AND RATES FROM .H5 DATA STORE:__
~~~
h5_store = h5py.File(h5_file, 'r')
print(f'Groups/Datasets: {list(h5_store.keys())}')

gids = np.array(h5_store['gids'])
firing_rates = np.array(h5_store['firing_rates'])

h5_store.close()
~~~